In [1]:
import logging
import datetime
#import random

import matplotlib.pyplot as plt

import numpy as np
import torch
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import config
import modules.dataloaders as data_loaders
import modules.utils as utils
import modules.models_original_Tinyissimo as models
import modules.loss as loss_module
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


# Logger

In [2]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('Original Tinyissimo Detector.\n' + 
            '\tNo Sigmoid, No Softmax. Permute out of the model.\n' +
            '\tOnly DFire.\n')

# Hyperparameters Log

In [3]:
''' ============================
    Print Config Values
============================ '''
print('\nDatasets Length')
print(f'\tTrain: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'\tVal: {"Full" if config.VAL_DS_LEN == None else config.VAL_DS_LEN}')
print(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    print(f'\tModel: {config.LOAD_MODEL_FILE}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate: {config.LEARNING_RATE}')
print(f'\tGradients Clip Norm: {config.GRADIENTS_CLIP_NORM}')
print(f'\tWeight Decay: {config.WEIGHT_DECAY}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs: {config.EPOCHS}')
print('IMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
print('\nGrid, Bounding Boxes, Classes, Max Obj and Thresholds:')
print(f'\tGrid: {config.S}')
print(f'\tNumber of Bounding Boxes per Cell: {config.B}')
print(f'\tNumber of Classes: {config.C}')
print(f'\tMaximum Number of Objects per Image: {config.MAX_OBJ}')
print(f'\tIOU Threshold: {config.IOU_THRESHOLD}')
print(f'\tScore Threshold: {config.SCORE_THRESHOLD}')

logger.info('\nDatasets Length')
logger.info(f'\tTrain: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\tVal: {"Full" if config.VAL_DS_LEN == None else config.VAL_DS_LEN}')
logger.info(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    logger.info(f'\tModel: {config.LOAD_MODEL_FILE}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate: {config.LEARNING_RATE}')
logger.info(f'\tGradients Clip Norm: {config.GRADIENTS_CLIP_NORM}')
logger.info(f'\tWeight Decay: {config.WEIGHT_DECAY}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs: {config.EPOCHS}')
logger.info('IMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
logger.info('\nGrid, Bounding Boxes, Classes and Thresholds:')
logger.info(f'\tGrid: {config.S}')
logger.info(f'\tNumber of Bounding Boxes per Cell: {config.B}')
logger.info(f'\tNumber of Classes: {config.C}')
logger.info(f'\tMaximum Number of Objects per Image: {config.MAX_OBJ}')
logger.info(f'\tIOU Threshold: {config.IOU_THRESHOLD}')
logger.info(f'\tScore Threshold: {config.SCORE_THRESHOLD}\n')


Datasets Length
	Train: Full
	Val: Full

Load Model: False
Device: cuda
Optimizer:
	Learning Rate: 0.001
	Gradients Clip Norm: 500
	Weight Decay: 0.001
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 3
	Scheduler threshold: 0.01
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 8
Pin Memory: True
Epochs: 100
IMG DIMS:
	Width: 88
	Height: 88

Grid, Bounding Boxes, Classes, Max Obj and Thresholds:
	Grid: 4
	Number of Bounding Boxes per Cell: 2
	Number of Classes: 2
	Maximum Number of Objects per Image: 10
	IOU Threshold: 0.5
	Score Threshold: 0.2


# Datasets and Dataloaders

In [4]:
train_loader = data_loaders.get_dfire_train_loader()
val_loader = data_loaders.get_dfire_val_loader()

# train_loader = data_loaders.get_fasdd_uav_train_loader()
# val_loader = data_loaders.get_fasdd_uav_val_loader()


TRAIN DFIRE dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 2228
DFire Removed due to more than 10: 59

Train DFire dataset len: 14934

TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 525
DFire Removed due to more than 10: 13

Test dataset len: 3768


### Plot Some Train Pictures

In [5]:
# plt.subplots(4, 5, figsize=(10,8))
# for i in range(20):
# plt.subplot(4, 5, i+1)

for batch_idx, (img, label) in enumerate(train_loader):
       
    if batch_idx == 0:
        print(f'Batch size equal to img.shape[0] = {img.shape[0]}')
        print(f'Batch images shape = {img.shape}')
        plt.subplots(4, 5, figsize=(10,8))
        for i in range(20):
            pic = utils.plot_dataset_img(img[i], label[i], grid=True)
            plt.subplot(4, 5, i+1)
            plt.imshow(pic)
        plt.tight_layout()
        plt.savefig(config.RUN_FOLDER + 'train_pictures.png')
        plt.close()
        break

Batch size equal to img.shape[0] = 64
Batch images shape = torch.Size([64, 3, 88, 88])


### Plot Some Val Pictures

In [6]:
# plt.subplots(4, 5, figsize=(10,8))
# for i in range(20):
# plt.subplot(4, 5, i+1)

for batch_idx, (img, label) in enumerate(val_loader):
       
    if batch_idx == 33:
        print(f'Batch size equal to img.shape[0] = {img.shape[0]}')
        print(f'Batch images shape = {img.shape}')
        plt.subplots(4, 5, figsize=(10,8))
        for i in range(20):
            pic = utils.plot_dataset_img(img[i], label[i], grid=True)
            plt.subplot(4, 5, i+1)
            plt.imshow(pic)
        plt.tight_layout()
        plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
        plt.close()
        break

Batch size equal to img.shape[0] = 64
Batch images shape = torch.Size([64, 3, 88, 88])


# Loss Setup

In [7]:
if config.LOSS_FN == "YOLOV1_LOSS":
    print(f'Loss Function: YOLOV1_LOSS')
    logger.info(f'\nLoss Function: YOLOV1_LOSS')
    loss_fn = loss_module.YoloLoss_2BBox()
    print(f'Lambda for L1 regularization: {config.LAMBDA_L1_LOSS}')
    logger.info(f'Lambda for L1 regularization: {config.LAMBDA_L1_LOSS}')
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: YOLOV1_LOSS
Lambda for L1 regularization: 0


# Model Setup

In [8]:
if config.MODEL == "Tinyissimo":
    
    print("Using Tinyissimo Detector")
    logger.info("\nUsing Tinyissimo Detector")
    model = models.Tinyissimo().to(config.DEVICE) 

else:
    print("Wrong Model")
    logger.info("Wrong Model")
    raise SystemExit("Wrong Model")

optimizer = optim.Adam(model.parameters(), 
                       lr=config.LEARNING_RATE, 
                       weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)

# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}')

Using Tinyissimo Detector

Trainable parameters = 468880
Total parameters = 468880



### Load Checkpoint

In [9]:
# epochs_trained = utils.load_checkpoint(config.LOAD_MODEL_FILE, 
#                                        model, 
#                                        optimizer=optimizer, 
#                                        scheduler=scheduler, 
#                                        device=config.DEVICE)

# logger.info(f"Loading Model. Trained during {epochs_trained} epochs")

### Check Model Shape

In [10]:
in_rand_np = np.random.rand(4, 3, config.IMG_H, config.IMG_W)
in_rand = torch.tensor(in_rand_np, dtype=torch.float32, device=config.DEVICE)
out_test = model(in_rand)

print(f'Input shape is {in_rand.shape}')
print(f'Model shape is {out_test.shape}')
print(f'BED Model Arquitecture\n{model}')
logger.info(f'\nInput shape is {in_rand.shape}')
logger.info(f'Model shape is {out_test.shape}\n')
logger.info(f'BED Model Arquitecture\n{model}')

Input shape is torch.Size([4, 3, 88, 88])
Model shape is torch.Size([4, 12, 4, 4])
BED Model Arquitecture
Tinyissimo(
  (tiny_model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False)
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False)
    (11): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13)

### Torch Summary

In [11]:
print(summary(model, input_size=(1, 3, config.IMG_H, config.IMG_W)))
logger.info("\nModel Summary")
logger.info(summary(model, input_size=(1, 3, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                   Output Shape              Param #
Tinyissimo                               [1, 12, 4, 4]             --
├─Sequential: 1-1                        [1, 12, 4, 4]             --
│    └─Conv2d: 2-1                       [1, 16, 88, 88]           432
│    └─BatchNorm2d: 2-2                  [1, 16, 88, 88]           32
│    └─ReLU: 2-3                         [1, 16, 88, 88]           --
│    └─Conv2d: 2-4                       [1, 16, 88, 88]           2,304
│    └─BatchNorm2d: 2-5                  [1, 16, 88, 88]           32
│    └─ReLU: 2-6                         [1, 16, 88, 88]           --
│    └─MaxPool2d: 2-7                    [1, 16, 44, 44]           --
│    └─Conv2d: 2-8                       [1, 16, 44, 44]           2,304
│    └─BatchNorm2d: 2-9                  [1, 16, 44, 44]           32
│    └─ReLU: 2-10                        [1, 16, 44, 44]           --
│    └─Conv2d: 2-11                      [1, 32, 44, 44]           4,608
│    

# Loss and Metrics Loggers and Plotters

In [12]:
train_losses_logger = utils.LogLosses()
train_metrics_logger = utils.LogMetrics()
lr_logger = utils.LogLR(log_path=config.PLOTS_FOLDER)

val_losses_logger = utils.LogLosses()
val_metrics_logger = utils.LogMetrics()

loss_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Loss')
metrics_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Metric')

# Train Loop Function

In [13]:
def train_loop(model, start_epoch=0, epochs_to_train=config.EPOCHS):

    ''' ==============================================================
                                TRAINING LOOP
    ============================================================== '''
    start = datetime.datetime.now()
    start_time = start.strftime("%H:%M:%S")
    print(f'\n***Start Training: {start_time}\n')
    logger.info(f'\n***Start Training: {start_time}\n')
    
    # Start with infinite validation loss
    best_valid_loss = np.inf
    best_mAP = torch.tensor(0., dtype=torch.float32)

    epochs_loss_plot = []
    epochs_metric_plot = []
    
    end_epoch = start_epoch + epochs_to_train
        
    for epoch in range(start_epoch, end_epoch):

        print(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        logger.info(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        
        #====================== TRAINING ========================#
        current_lr = train_epoch.get_lr(optimizer=optimizer)
        logger.info(f'Learning Rate = {current_lr}\n')
        lr_logger.log_lr(current_lr)
            
        calculate_mAP = False
        if ( (epoch+1) % 5 ) == 0:
            calculate_mAP = True
            epochs_metric_plot.append(epoch)

        train_losses, train_metrics = train_epoch.train_fn(
            loader=train_loader, 
            model=model, 
            optimizer=optimizer, 
            loss_fn=loss_fn,
            loss_l1_lambda=config.LAMBDA_L1_LOSS,
            metric=metrics.map_metric,
            device=config.DEVICE,
            calculate_mAP=calculate_mAP)
        
        train_losses_logger.update_losses(train_losses)
        if calculate_mAP == True:
            train_metrics_logger.update_metrics(train_metrics)
                
        logger.info(utils.print_metrics_to_logger("TRAIN STATS", train_losses, train_metrics, mAP_available=calculate_mAP))
        
        #===================== VALIDATING =======================#
        with torch.no_grad():
            val_losses, val_metrics = val_epoch.eval_fn(
                loader=val_loader, 
                model=model,                         
                loss_fn=loss_fn,
                metric=metrics.map_metric,
                device=config.DEVICE,
                calculate_mAP=calculate_mAP)
            
            scheduler.step(val_losses['Total'])
            
            val_losses_logger.update_losses(val_losses)
            if calculate_mAP == True:
                val_metrics_logger.update_metrics(val_metrics)

            logger.info(utils.print_metrics_to_logger("VAL STATS", val_losses, val_metrics, mAP_available=calculate_mAP))
            
        epochs_loss_plot.append(epoch)

        loss_plotter.plot_all_metrics(
            train_losses_logger.get_losses(),
            val_losses_logger.get_losses(),
            epochs_loss_plot)

        if calculate_mAP == True:
            metrics_plotter.plot_all_metrics(
                train_metrics_logger.get_metrics(),
                val_metrics_logger.get_metrics(),
                epochs_metric_plot)

        lr_logger.plot_lr(epochs_loss_plot)
        
        #======================= SAVING =========================#
        if ( (epoch+1) % 5 ) == 0:
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__5epoch.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            
        if best_valid_loss > val_losses['Total']:
            best_valid_loss = val_losses['Total']
            print(f"\nSaving model with new best validation loss: {best_valid_loss:.3f}")
            logger.info(f"Saving model with new best validation loss: {best_valid_loss:.3f}")
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__' + 'best_loss'  + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  

        # Save model if mAP increases
        if calculate_mAP == True:
            if ( best_mAP < val_metrics['mAP'] ) :
                best_mAP = val_metrics['mAP']
                print(f"\nSaving model with new best mAP: {best_mAP:.4f}")
                logger.info(f"Saving model with new best mAP: {best_mAP:.4f}")
                save_precision_name = f'best_mAP={best_mAP:.4f}__epoch={epoch}'
                save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__' + save_precision_name + '.pt'
                utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  
        
    logger.info('Saving last model')   
    torch.save(model.state_dict(), config.WEIGHTS_FOLDER + 'last_' + config.MODEL + '_detector.pt') 
    
    #======================= FINISH =========================#
    end = datetime.datetime.now()
    end_time = end.strftime("%H:%M:%S")
    print(f'\n***Script finished: {end_time}\n')  
    print(f'Time elapsed: {end-start}')
    logger.info(f'\n***Script finished: {end_time}\n')  
    logger.info(f'Time elapsed: {end-start}')
    
    return model

# Main Execute

In [14]:
print("Starting script\n")
logger.info("Starting script\n")
    
model = train_loop(model)

Starting script


***Start Training: 16:41:11


=== EPOCH 0/99 ===
Learning Rate = 0.001



Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 233/233 [00:12<00:00, 18.27it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
83.177      |42.647      |21.619      |8.025       |10.886      


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:01<00:00, 34.04it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
127.538     |70.492      |37.748      |1.761       |17.537      

Saving model with new best validation loss: 127.538

=== EPOCH 1/99 ===
Learning Rate = 0.001



...

Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
84.049      |41.396      |29.203      |2.581       |10.869      

=== EPOCH 99/99 ===
Learning Rate = 0.00013421772800000008



Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 233/233 [00:15<00:00, 14.89it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
26.022      |9.487       |11.064      |3.274       |2.197       
Train mAP = 0.3518


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:02<00:00, 21.90it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
83.718      |41.687      |28.926      |2.636       |10.469      
Val mAP = 0.2234

Saving model with new best mAP: 0.2234

***Script finished: 17:09:11

Time elapsed: 0:28:00.171408


# mAP with Aladdin to modify threshold and draw PR curve

In [15]:
import modules.metrics_mAP_aladdin as aladdin_mAP

In [16]:
def print_metrics(mAP_metrics):
    mAP, avg_prec, cls_prec, cls_rec = mAP_metrics
    
    mAP_str = "mAP @0.50"
    smoke = "Smoke"
    fire = "Fire"
    
    print(f'{mAP_str:<12}' + f'{mAP:.4f}')
    print('Average Precision')
    print(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    print('Class Precision')
    print(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    print('Class Recall')
    print(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')
    
    logger.info(f'{mAP_str:<12}' + f'{mAP:.4f}')
    logger.info('Average Precision')
    logger.info(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    logger.info('Class Precision')
    logger.info(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    logger.info('Class Recall')
    logger.info(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')

In [17]:
model.eval()
print("Model changed to evaluation")

Model changed to evaluation


In [18]:
pred_boxes_1, true_boxes_1 = aladdin_mAP.get_bboxes(
    loader=val_loader,
    model=model,
    threshold=0.2)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:02<00:00, 24.29it/s]


In [19]:
save_path = './experiments_originals/10_tinyissimo_original/'
dfire_metrics = aladdin_mAP.mAP(
    log_path=save_path,
    pred_boxes=pred_boxes_1,
    true_boxes=true_boxes_1)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.15it/s]


In [20]:
logger.info('\n\nDFire Aladdin Metrics:\n')
print_metrics(dfire_metrics)

mAP @0.50   0.2161
Average Precision
- Smoke     0.3684
- Fire      0.0639
Class Precision
- Smoke     0.6015
- Fire      0.3003
Class Recall
- Smoke     0.4426
- Fire      0.1165


In [21]:
pred_boxes_2, true_boxes_2 = aladdin_mAP.get_bboxes(
    loader=val_loader,
    model=model,
    threshold=0.001)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:17<00:00,  3.35it/s]


In [22]:
dfire_metrics = aladdin_mAP.mAP(
    log_path=save_path,
    pred_boxes=pred_boxes_2,
    true_boxes=true_boxes_2)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.61s/it]


In [23]:
logger.info('\n\nDFire Aladdin Metrics: threshold changed to 0.001\n')
print_metrics(dfire_metrics)

mAP @0.50   0.2308
Average Precision
- Smoke     0.3920
- Fire      0.0696
Class Precision
- Smoke     0.0234
- Fire      0.0368
Class Recall
- Smoke     0.5112
- Fire      0.1488


## Fire has terrible statistics. Change IOU thres to check

In [24]:
dfire_metrics = aladdin_mAP.mAP(
    log_path=save_path,
    pred_boxes=pred_boxes_1,
    true_boxes=true_boxes_1,
    iou_threshold=0.3)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.09it/s]


In [25]:
logger.info('\n\nDFire Aladdin Metrics: threshold changed back to 0.2 and IOU to 0.3\n')
print_metrics(dfire_metrics)

mAP @0.50   0.3265
Average Precision
- Smoke     0.4844
- Fire      0.1686
Class Precision
- Smoke     0.7170
- Fire      0.5389
Class Recall
- Smoke     0.5276
- Fire      0.2092


In [26]:
dfire_metrics = aladdin_mAP.mAP(
    log_path=save_path,
    pred_boxes=pred_boxes_2,
    true_boxes=true_boxes_2,
    iou_threshold=0.3)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.63s/it]


In [27]:
logger.info('\n\nDFire Aladdin Metrics: threshold 0.001 again and IOU to 0.3\n')
print_metrics(dfire_metrics)

mAP @0.50   0.3609
Average Precision
- Smoke     0.5264
- Fire      0.1954
Class Precision
- Smoke     0.0290
- Fire      0.0709
Class Recall
- Smoke     0.6333
- Fire      0.2862
